In [1]:
import os
import time
import re
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
import spacy
import pickle
nltk.download('stopwords')
import unidecode
from sklearn.utils import shuffle
import sklearn.metrics as met
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF

2022-12-20 04:35:02.453826: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-20 04:35:02.453903: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-20 04:35:02.453913: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-12-20 04:35:03.108121: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-1

In [2]:
from IPython.display import display,HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth",200)

In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
startTime = time.time()

# Data

In [5]:
path = '/home/thomas/Documents/openclassroom/projet5/'
path_res = path +'results/'

In [6]:
with open(path + 'cleanDataprepSmote3.pickle','rb') as f:
    data = pickle.load(f)

In [7]:
data.shape

(51904, 20)

In [8]:
data.head()

,Title,Body,Tags,text,nb_char,tokens,tokens_title,nb_tokens,tokens_unique,nb_tokens_unique,tokens_WSW,tokens_title_WSW,nb_tokens_WSW,tokens_lemma,len_tokens_lemma,Tags_list,len _Tags_list,filtered_tags,filtered_tags_reduced,len_filtered_tags_reduced
0,transpose/unzip function (inverse of zip)?,<p>I have a list of 2-item tuples and I'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.</p>\n\n<p><strong>For exa...,<python><list><matrix><transpose>,i have a list of 2-item tuples and i'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.\nfor example:\noriginal = [(...,350,"[i, have, a, list, of, 2, item, tuples, and, i, d, like, to, convert, them, to, 2, lists, where, the, first, contains, the, first, item, in, each, tuple, and, the, second, list, holds, the, second...","[transpose, unzip, function, inverse, of, zip]",70,"(have, d, list, and, result, to, lists, c, convert, second, example, 2, for, original, 4, holds, want, builtin, #, there, item, them, that, tuples, contains, where, first, in, function, does, like...",42,"[list, item, tuples, like, convert, lists, first, contains, first, item, tuple, second, list, holds, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]","[transpose, unzip, function, inverse, zip]",28,"[list, item, tuple, like, convert, list, first, contain, first, item, tuple, second, list, hold, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]",28,"[list, transpose, matrix, python]",4,"[list, python]",tag0,1
1,detecting audio silence in wav files using c#,"<p>I'm tasked with building a .NET client app to detect silence in a WAV files.</p>\n\n<p>Is this possible with the built-in Windows APIs? Or alternately, any good libraries out there to help with...",<c#><.net><audio>,"i'm tasked with building a .net client app to detect silence in a wav files.\nis this possible with the built-in windows apis? or alternately, any good libraries out there to help with this?\n",190,"[i, m, tasked, with, building, a, net, client, app, to, detect, silence, in, a, wav, files, is, this, possible, with, the, built, in, windows, apis, or, alternately, any, good, libraries, out, the...","[detecting, audio, silence, in, wav, files, using, c#]",36,"(good, tasked, silence, to, windows, app, files, this, help, alternately, any, there, is, wav, built, detect, m, in, building, possible, libraries, or, apis, out, with, a, the, i, net, client)",30,"[tasked, building, net, client, app, detect, silence, wav, files, possible, built, windows, apis, alternately, good, libraries, help]","[detecting, audio, silence, wav, files, using, c#]",17,"[task, build, net, client, app, detect, silence, wav, file, possible, build, window, apis, alternately, good, library, help]",17,"[c#, audio, .net]",3,"[c#, .net]",tag1,1
2,how to request a random row in sql?,<p>How can I request a random row (or as close to truly random as is possible) in pure SQL?</p>\n,<sql><random>,how can i request a random row (or as close to truly random as is possible) in pure sql?\n,89,"[how, can, i, request, a, random, row, or, as, close, to, truly, random, as, is, possible, in, pure, sql]","[how, to, request, a, random, row, in, sql]",19,"(a, row, how, i, sql, in, to, close, can, truly, request, possible, is, or, random, pure, as)",17,"[request, random, row, close, truly, random, possible, pure, sql]","[request, random, row, sql]",9,"[request, random, row, close, truly, random, possible, pure, sql]",9,"[random, sql]",2,[sql],tag108,1
4,detecting current iphone input language,"<p>Does anybody knows, can I get the current input language and/or keyboard layout in iPhone application? Can I also get a notification when input language was changed?</p>\n",<iphone><keyboard>,"does anybody knows, can i get the current input language and/or keyboard layout in

In [9]:
data.dtypes

Title                        object
Body                         object
Tags                         object
text                         object
nb_char                       int64
tokens                       object
tokens_title                 object
nb_tokens                     int64
tokens_unique                object
nb_tokens_unique              int64
tokens_WSW                   object
tokens_title_WSW             object
nb_tokens_WSW                 int64
tokens_lemma                 object
len_tokens_lemma              int64
Tags_list                    object
len _Tags_list                int64
filtered_tags                object
filtered_tags_reduced        object
len_filtered_tags_reduced     int64
dtype: object

## prepare data

list all empty filtered_tags

In [10]:
data['filtered_tags'][data['filtered_tags'].str.len() == 0]

Series([], Name: filtered_tags, dtype: object)

In [11]:
data['filtered_tags'] = data['filtered_tags'].apply(lambda x : ['#empty'] if len(''.join(x)) == 0 else x)

In [12]:
data['filtered_tags_reduced'][data['filtered_tags_reduced'].str.len() == 0]

Series([], Name: filtered_tags_reduced, dtype: object)

In [13]:
data['filtered_tags_reduced'] = data['filtered_tags_reduced'].apply(lambda x : ['#empty'] if len(''.join(x)) == 0 else x)

In [14]:
data['filtered_tags_reduced'][data['filtered_tags_reduced'].str.len() == 0]

Series([], Name: filtered_tags_reduced, dtype: object)

In [15]:
print(data.shape)

(51904, 20)


In [16]:
data.head(8)

,Title,Body,Tags,text,nb_char,tokens,tokens_title,nb_tokens,tokens_unique,nb_tokens_unique,tokens_WSW,tokens_title_WSW,nb_tokens_WSW,tokens_lemma,len_tokens_lemma,Tags_list,len _Tags_list,filtered_tags,filtered_tags_reduced,len_filtered_tags_reduced
0,transpose/unzip function (inverse of zip)?,<p>I have a list of 2-item tuples and I'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.</p>\n\n<p><strong>For exa...,<python><list><matrix><transpose>,i have a list of 2-item tuples and i'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.\nfor example:\noriginal = [(...,350,"[i, have, a, list, of, 2, item, tuples, and, i, d, like, to, convert, them, to, 2, lists, where, the, first, contains, the, first, item, in, each, tuple, and, the, second, list, holds, the, second...","[transpose, unzip, function, inverse, of, zip]",70,"(have, d, list, and, result, to, lists, c, convert, second, example, 2, for, original, 4, holds, want, builtin, #, there, item, them, that, tuples, contains, where, first, in, function, does, like...",42,"[list, item, tuples, like, convert, lists, first, contains, first, item, tuple, second, list, holds, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]","[transpose, unzip, function, inverse, zip]",28,"[list, item, tuple, like, convert, list, first, contain, first, item, tuple, second, list, hold, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]",28,"[list, transpose, matrix, python]",4,"[list, python]",tag0,1
1,detecting audio silence in wav files using c#,"<p>I'm tasked with building a .NET client app to detect silence in a WAV files.</p>\n\n<p>Is this possible with the built-in Windows APIs? Or alternately, any good libraries out there to help with...",<c#><.net><audio>,"i'm tasked with building a .net client app to detect silence in a wav files.\nis this possible with the built-in windows apis? or alternately, any good libraries out there to help with this?\n",190,"[i, m, tasked, with, building, a, net, client, app, to, detect, silence, in, a, wav, files, is, this, possible, with, the, built, in, windows, apis, or, alternately, any, good, libraries, out, the...","[detecting, audio, silence, in, wav, files, using, c#]",36,"(good, tasked, silence, to, windows, app, files, this, help, alternately, any, there, is, wav, built, detect, m, in, building, possible, libraries, or, apis, out, with, a, the, i, net, client)",30,"[tasked, building, net, client, app, detect, silence, wav, files, possible, built, windows, apis, alternately, good, libraries, help]","[detecting, audio, silence, wav, files, using, c#]",17,"[task, build, net, client, app, detect, silence, wav, file, possible, build, window, apis, alternately, good, library, help]",17,"[c#, audio, .net]",3,"[c#, .net]",tag1,1
2,how to request a random row in sql?,<p>How can I request a random row (or as close to truly random as is possible) in pure SQL?</p>\n,<sql><random>,how can i request a random row (or as close to truly random as is possible) in pure sql?\n,89,"[how, can, i, request, a, random, row, or, as, close, to, truly, random, as, is, possible, in, pure, sql]","[how, to, request, a, random, row, in, sql]",19,"(a, row, how, i, sql, in, to, close, can, truly, request, possible, is, or, random, pure, as)",17,"[request, random, row, close, truly, random, possible, pure, sql]","[request, random, row, sql]",9,"[request, random, row, close, truly, random, possible, pure, sql]",9,"[random, sql]",2,[sql],tag108,1
4,detecting current iphone input language,"<p>Does anybody knows, can I get the current input language and/or keyboard layout in iPhone application? Can I also get a notification when input language was changed?</p>\n",<iphone><keyboard>,"does anybody knows, can i get the current input language and/or keyboard layout in

# models

## shared functions

In [17]:
def wordFrequency(data):
    ''' take pandas series and count words retun the occurences of the elements'''
    freqTotal = {}
    for text in data:
        freq = nltk.Counter(text)
        for word, count in freq.items():
            if word in freqTotal.keys():
                freqTotal[word] += count
            else:
                freqTotal[word] = count
    return freqTotal

def splitData(df,XCol,Ycol,ratioToKeep=1):
    """ split data into two sets X and Y and then split them into  trainning and validation sets 
    the total will be 4 sets. the function returns pandas series or pandas dataframe types"""# add sample 
    df = df.sample(frac=ratioToKeep)
    print(f'data used shape: {df.shape}')
    X=df.loc[:,XCol]
    y=df.loc[:,Ycol]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    print(f'X_train types:\t{type(X_train)}')
    print(f'X_test types:\t{type(X_test)}')
    print(f'Y_test types:\t{type(y_test)}')
    print(f'Y_train types:\t{type(y_train)}')
    return X_train, X_test, y_train, y_test

def reduceData (Xtrain, Ytrain, ratioToKeep=0.2):
    size = Xtrain.shape[0]
    nbRowsToKeep = int(np.rint(size*ratioToKeep))
    indRowToKeep = np.random.choice(size, nbRowsToKeep)
    Xtrain = Xtrain[indRowToKeep,:]
    Ytrain = Ytrain[indRowToKeep,:]
    return (Xtrain,Ytrain)

# BOW

## functions

In [18]:
def BOW(Xtrain, Xval):
    """ set the BOW method for X train and validation sets."""
    Xtrain = Xtrain.apply(lambda x : ' '.join(x))
    Xval = Xval.apply(lambda x : ' '.join(x))
    XtrainCorpus = [x for x in Xtrain]
    XvalCorpus = [x for x in Xval]
    vectorizer = CountVectorizer()
    Xtrain = vectorizer.fit_transform(XtrainCorpus)
    print('-'*15)
    print(f"Xtrain vectorized shape: {Xtrain.shape}\n")
    Xval = vectorizer.transform(XvalCorpus)
    print(f"Xval vectorized shape: {Xval.shape}\n")
    return (Xtrain,Xval)


def Binarize(Ytrain,Yval):
    """  binarize The tags for Y train and validation sets. """
    mlb = MultiLabelBinarizer()
    mlb.fit([Ytrain]) # if Y is not list of list, mlb.fit(['sci-fi', 'thriller', 'comedy']) -> array(['-', 'c', 'd', 'e', 'f', 'h', 'i', 'l', 'm', 'o', 'r', 's', 't','y'], dtype=object)
    for i in Ytrain.index:
        Ytrain.loc[i] = mlb.transform([[Ytrain.loc[i]]])[0]
    for i in Yval.index:
        Yval.loc[i] = mlb.transform([[Yval.loc[i]]])[0]
    labels = mlb.classes_
    print('-'*15)
    print(f"Ytrain vectorized shape: {Ytrain.shape}\n")
    print(f"Yval vectorized shape: {Yval.shape}\n")
    print(f"classes : {mlb.classes_}")
    return (Ytrain,Yval, labels)

def plotRoc():
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(labels)):
        fpr[i], tpr[i], _ = roc_curve(y_val[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    plt.figure()
    lw = 2
    plt.plot(
        fpr[2],
        tpr[2],
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc[2],
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()





def clfScore(model,Xval, Yval, labels, fileName,PATH=path_res):                           #  correct position of the labels ???
    y_pred = model.predict(Xval)
    F1score=met.f1_score(y_pred, Yval, average=None)
    preciScore=met.precision_score(y_pred, Yval, average=None)
    recallScore=met.recall_score(y_pred, Yval, average=None)
    print(f'{"Tags":25}\t\t F-score\t\t precision\t\t recall\n')
    with open(PATH + fileName,'w') as f:
        for ind,name in enumerate(labels):
            f.write(f'{name:25}\t\t {F1score[ind]:4f}\t\t {preciScore[ind]:4f}\t\t {recallScore[ind]:4f}\n')
            print(f'{name:25}\t\t {F1score[ind]:4f}\t\t {preciScore[ind]:4f}\t\t {recallScore[ind]:4f}')
    #plotRoc(labels,y_pred,y_val)
    
        

    
    

 ## logistic regression

### one VS rest

#### original tags

#### Grouped tags

In [19]:
X_train, X_val, y_train, y_val = splitData(data,XCol='tokens_lemma',Ycol='filtered_tags_reduced',ratioToKeep=1) # too much time without sampling
X_train, X_val = BOW(X_train, X_val)
y_train, y_val, labels = Binarize(y_train,y_val)

y_train = [ i for i in y_train]
y_val = [ i for i in y_val]
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)


print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")

oversample = SMOTE(random_state=42)
X_train, y_train = oversample.fit_resample(X_train, y_train) # SMOTE
X_train, y_train = shuffle(X_train,y_train, random_state=1)
print(f"---- After SMOTE ----")
print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")

print(f"---- Reduce size training set ----")
X_train, y_train = reduceData(X_train ,y_train,ratioToKeep=0.2) 

print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")
print(f"X_val vectorized shape: {X_val.shape}\n")


log_clf = OneVsRestClassifier(LogisticRegression(random_state=42,
                                                 max_iter=10000,
                                                 verbose=0))

model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',log_clf)])
#raise Exception("trop long") 
# hyperparameters
param_grid = [{'estimator__estimator__C': [0.1, 0.5, 1., 5, 10]}] # lambda regularization

# cross-validation
clf = GridSearchCV(model,
                   param_grid,
                   scoring='f1_macro', #F1 = 2 * (precision * recall) / (precision + recall)
                   n_jobs=-1,
                   verbose=0,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train, y_train)
log_clf_best_param = clf.best_params_['estimator__estimator__C']
print('Meilleurs hyper-paramètres :')
print(clf.best_params_)

data used shape: (51904, 20)
X_train types:	<class 'pandas.core.series.Series'>
X_test types:	<class 'pandas.core.series.Series'>
Y_test types:	<class 'pandas.core.series.Series'>
Y_train types:	<class 'pandas.core.series.Series'>
---------------
Xtrain vectorized shape: (34775, 67652)

Xval vectorized shape: (17129, 67652)

---------------
Ytrain vectorized shape: (34775,)

Yval vectorized shape: (17129,)

classes : ['tag0' 'tag1' 'tag10' 'tag108' 'tag11' 'tag12' 'tag13' 'tag14' 'tag15'
 'tag16' 'tag17' 'tag18' 'tag19' 'tag2' 'tag20' 'tag21' 'tag22' 'tag23'
 'tag24' 'tag25' 'tag26' 'tag27' 'tag28' 'tag29' 'tag3' 'tag30' 'tag31'
 'tag32' 'tag33' 'tag34' 'tag35' 'tag36' 'tag37' 'tag38' 'tag4' 'tag5'
 'tag6' 'tag7' 'tag8' 'tag9']
y_train vectorized shape: (34775, 40)

y_val vectorized shape: (17129, 40)

---- After SMOTE ----
y_train vectorized shape: (141360, 40)

X_train vectorized shape: (141360, 67652)

---- Reduce size training set ----
y_train vectorized shape: (28272, 40)

y_val v

In [20]:
log_clf = OneVsRestClassifier(LogisticRegression(random_state=42,
                                                 max_iter=10000,
                                                 verbose=0,
                                                C=log_clf_best_param))
model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',log_clf)])
model.fit(X_train, y_train)
clfScore(model, X_val,y_val, labels,'BOW_logistic_reduced_without_empty')

Tags                     		 F-score		 precision		 recall

tag0                     		 0.392497		 0.337313		 0.469269
tag1                     		 0.234321		 0.201183		 0.280528
tag10                    		 0.376543		 0.439904		 0.329137
tag108                   		 0.354874		 0.315789		 0.405000
tag11                    		 0.195599		 0.185615		 0.206718
tag12                    		 0.758133		 0.722372		 0.797619
tag13                    		 0.136072		 0.157738		 0.119639
tag14                    		 0.263815		 0.254296		 0.274074
tag15                    		 0.182965		 0.207143		 0.163842
tag16                    		 0.178010		 0.188192		 0.168874
tag17                    		 0.259965		 0.294118		 0.232919
tag18                    		 0.128244		 0.175000		 0.101205
tag19                    		 0.304029		 0.344398		 0.272131
tag2                     		 0.357354		 0.290809		 0.463388
tag20                    		 0.139918		 0.142857		 0.137097
tag21                    		 0.214411		 0.262931		 0.18100

## Support vector classification

### one VS rest

#### original tags

#### Grouped tags

## Random Forest

### one VS rest

#### original tags

#### Grouped tags

In [21]:
X_train, X_val, y_train, y_val = splitData(data,XCol='tokens_lemma',Ycol='filtered_tags_reduced',ratioToKeep=1) # too much time without sampling
X_train, X_val = BOW(X_train, X_val)
y_train, y_val, labels = Binarize(y_train,y_val)

y_train = [ i for i in y_train]
y_val = [ i for i in y_val]
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)


print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")

oversample = SMOTE(random_state=42)
X_train, y_train = oversample.fit_resample(X_train, y_train) # SMOTE
X_train, y_train = shuffle(X_train,y_train, random_state=1)
print(f"---- After SMOTE ----")
print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")

print(f"---- Reduce size training set ----")
X_train, y_train = reduceData(X_train ,y_train,ratioToKeep=0.05) 

print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")
print(f"X_val vectorized shape: {X_val.shape}\n")


RF_clf = OneVsRestClassifier(RF(min_samples_split=5))
model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',RF_clf)])
#raise Exception("trop long") 
# hyperparameters
param_grid = [{'estimator__estimator__n_estimators': [10,20,30],'estimator__estimator__max_depth' : [5,8,10,15]}] 

# cross-validation
clf = GridSearchCV(model,
                   param_grid,
                   scoring='f1_macro', #F1 = 2 * (precision * recall) / (precision + recall)
                   n_jobs=-1,
                   verbose=0,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train, y_train)
RF_clf_best_param1 = clf.best_params_['estimator__estimator__n_estimators']
RF_clf_best_param2 = clf.best_params_['estimator__estimator__max_depth']
print('Meilleurs hyper-paramètres :')
print(clf.best_params_)

data used shape: (51904, 20)
X_train types:	<class 'pandas.core.series.Series'>
X_test types:	<class 'pandas.core.series.Series'>
Y_test types:	<class 'pandas.core.series.Series'>
Y_train types:	<class 'pandas.core.series.Series'>
---------------
Xtrain vectorized shape: (34775, 68065)

Xval vectorized shape: (17129, 68065)

---------------
Ytrain vectorized shape: (34775,)

Yval vectorized shape: (17129,)

classes : ['tag0' 'tag1' 'tag10' 'tag108' 'tag11' 'tag12' 'tag13' 'tag14' 'tag15'
 'tag16' 'tag17' 'tag18' 'tag19' 'tag2' 'tag20' 'tag21' 'tag22' 'tag23'
 'tag24' 'tag25' 'tag26' 'tag27' 'tag28' 'tag29' 'tag3' 'tag30' 'tag31'
 'tag32' 'tag33' 'tag34' 'tag35' 'tag36' 'tag37' 'tag38' 'tag4' 'tag5'
 'tag6' 'tag7' 'tag8' 'tag9']
y_train vectorized shape: (34775, 40)

y_val vectorized shape: (17129, 40)

---- After SMOTE ----
y_train vectorized shape: (140800, 40)

X_train vectorized shape: (140800, 68065)

---- Reduce size training set ----
y_train vectorized shape: (7040, 40)

y_val ve

In [22]:
RF_clf = OneVsRestClassifier(RF(n_estimators=RF_clf_best_param1,
                                                  max_depth=RF_clf_best_param2,
                                                   min_samples_split=5))
model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',RF_clf)])
model.fit(X_train, y_train)
clfScore(model, X_val,y_val, labels,'BOW_RF_reduced_without_empty')

Tags                     		 F-score		 precision		 recall

tag0                     		 0.000000		 0.000000		 0.000000
tag1                     		 0.000000		 0.000000		 0.000000
tag10                    		 0.009975		 0.005076		 0.285714
tag108                   		 0.000000		 0.000000		 0.000000
tag11                    		 0.000000		 0.000000		 0.000000
tag12                    		 0.080808		 0.042105		 1.000000
tag13                    		 0.000000		 0.000000		 0.000000
tag14                    		 0.000000		 0.000000		 0.000000
tag15                    		 0.000000		 0.000000		 0.000000
tag16                    		 0.000000		 0.000000		 0.000000
tag17                    		 0.000000		 0.000000		 0.000000
tag18                    		 0.000000		 0.000000		 0.000000
tag19                    		 0.000000		 0.000000		 0.000000
tag2                     		 0.000000		 0.000000		 0.000000
tag20                    		 0.000000		 0.000000		 0.000000
tag21                    		 0.000000		 0.000000		 0.00000

/home/thomas/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
endTime = time.time()
print(f"Execution Time {endTime - startTime} s")

Execution Time 6674.787618160248 s
